# Marketing Campaign Exploratory Data Analysis

This notebook performs comprehensive EDA on marketing campaign data to uncover insights and patterns that inform the dashboard design.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## 1. Data Loading and Initial Exploration

In [ ]:
# Load datasets
daily_metrics = pd.read_csv('../data/daily_marketing_metrics.csv')
customer_data = pd.read_csv('../data/customer_data.csv')
ab_test_results = pd.read_csv('../data/ab_test_results.csv')

# Convert date columns
daily_metrics['date'] = pd.to_datetime(daily_metrics['date'])
customer_data['acquisition_date'] = pd.to_datetime(customer_data['acquisition_date'])

print("Dataset Shapes:")
print(f"Daily Metrics: {daily_metrics.shape}")
print(f"Customer Data: {customer_data.shape}")
print(f"A/B Test Results: {ab_test_results.shape}")

In [ ]:
# Basic statistics
print("Daily Metrics Summary:")
daily_metrics.info()
print("\nFirst 5 rows:")
daily_metrics.head()

In [ ]:
# Calculate derived metrics
daily_metrics['ctr'] = daily_metrics['clicks'] / daily_metrics['impressions']
daily_metrics['conversion_rate'] = daily_metrics['conversions'] / daily_metrics['clicks']
daily_metrics['cpc'] = daily_metrics['spend'] / daily_metrics['clicks']
daily_metrics['cpa'] = daily_metrics['spend'] / daily_metrics['conversions'].replace(0, 1)
daily_metrics['roas'] = daily_metrics['revenue'] / daily_metrics['spend']
daily_metrics['aov'] = daily_metrics['revenue'] / daily_metrics['conversions'].replace(0, 1)

# Summary statistics
summary_stats = daily_metrics.describe()
print("Summary Statistics:")
summary_stats

## 2. Overall Performance Analysis

In [ ]:
# Overall KPIs
total_spend = daily_metrics['spend'].sum()
total_revenue = daily_metrics['revenue'].sum()
total_conversions = daily_metrics['conversions'].sum()
overall_roas = total_revenue / total_spend
overall_cpa = total_spend / total_conversions

print("Overall Marketing KPIs:")
print(f"Total Spend: ${total_spend:,.2f}")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Total Conversions: {total_conversions:,}")
print(f"Overall ROAS: {overall_roas:.2f}x")
print(f"Overall CPA: ${overall_cpa:.2f}")
print(f"ROI: {((total_revenue - total_spend) / total_spend * 100):.1f}%")

In [ ]:
# Monthly trend analysis
daily_metrics['month'] = daily_metrics['date'].dt.to_period('M')
monthly_performance = daily_metrics.groupby('month').agg({
    'spend': 'sum',
    'revenue': 'sum',
    'conversions': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()

monthly_performance['roas'] = monthly_performance['revenue'] / monthly_performance['spend']
monthly_performance['month'] = monthly_performance['month'].astype(str)

# Create subplots for monthly trends
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Monthly Spend vs Revenue', 'Monthly ROAS Trend', 
                   'Monthly Conversions', 'Monthly CTR & Conversion Rate')
)

# Spend vs Revenue
fig.add_trace(go.Bar(x=monthly_performance['month'], y=monthly_performance['spend'],
                    name='Spend', marker_color='lightcoral'), row=1, col=1)
fig.add_trace(go.Bar(x=monthly_performance['month'], y=monthly_performance['revenue'],
                    name='Revenue', marker_color='lightgreen'), row=1, col=1)

# ROAS Trend
fig.add_trace(go.Scatter(x=monthly_performance['month'], y=monthly_performance['roas'],
                        mode='lines+markers', name='ROAS', line=dict(color='blue', width=3)),
             row=1, col=2)

# Conversions
fig.add_trace(go.Bar(x=monthly_performance['month'], y=monthly_performance['conversions'],
                    name='Conversions', marker_color='orange'), row=2, col=1)

# CTR and Conversion Rate
monthly_performance['ctr'] = monthly_performance['clicks'] / monthly_performance['impressions'] * 100
fig.add_trace(go.Scatter(x=monthly_performance['month'], y=monthly_performance['ctr'],
                        mode='lines+markers', name='CTR %', line=dict(color='purple')),
             row=2, col=2)

fig.update_layout(height=800, showlegend=True, title_text="Monthly Performance Overview")
fig.show()

## 3. Channel Performance Deep Dive

In [ ]:
# Channel performance metrics
channel_performance = daily_metrics.groupby('channel').agg({
    'spend': 'sum',
    'revenue': 'sum',
    'conversions': 'sum',
    'impressions': 'sum',
    'clicks': 'sum',
    'new_customers': 'sum'
}).reset_index()

# Calculate metrics
channel_performance['roas'] = channel_performance['revenue'] / channel_performance['spend']
channel_performance['cpa'] = channel_performance['spend'] / channel_performance['conversions']
channel_performance['ctr'] = channel_performance['clicks'] / channel_performance['impressions'] * 100
channel_performance['conversion_rate'] = channel_performance['conversions'] / channel_performance['clicks'] * 100
channel_performance['revenue_share'] = channel_performance['revenue'] / channel_performance['revenue'].sum() * 100

# Sort by ROAS
channel_performance = channel_performance.sort_values('roas', ascending=False)

print("Channel Performance Summary:")
display_cols = ['channel', 'spend', 'revenue', 'conversions', 'roas', 'cpa', 'ctr', 'revenue_share']
channel_performance[display_cols]

In [ ]:
# Channel performance visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# ROAS by Channel
channel_performance.plot(x='channel', y='roas', kind='bar', ax=axes[0, 0], color='skyblue')
axes[0, 0].set_title('ROAS by Channel', fontsize=14)
axes[0, 0].axhline(y=1, color='red', linestyle='--', label='Break-even')
axes[0, 0].set_ylabel('ROAS')

# Spend vs Revenue
x = channel_performance['spend']
y = channel_performance['revenue']
axes[0, 1].scatter(x, y, s=100)
for i, txt in enumerate(channel_performance['channel']):
    axes[0, 1].annotate(txt, (x.iloc[i], y.iloc[i]), fontsize=9)
axes[0, 1].set_xlabel('Spend ($)')
axes[0, 1].set_ylabel('Revenue ($)')
axes[0, 1].set_title('Spend vs Revenue by Channel', fontsize=14)

# Revenue Share Pie Chart
axes[1, 0].pie(channel_performance['revenue_share'], labels=channel_performance['channel'], 
               autopct='%1.1f%%', startangle=90)
axes[1, 0].set_title('Revenue Share by Channel', fontsize=14)

# CPA Comparison
channel_performance.plot(x='channel', y='cpa', kind='bar', ax=axes[1, 1], color='coral')
axes[1, 1].set_title('Cost Per Acquisition by Channel', fontsize=14)
axes[1, 1].set_ylabel('CPA ($)')

plt.tight_layout()
plt.show()

In [ ]:
# Channel trend analysis over time
channel_daily = daily_metrics.groupby(['date', 'channel']).agg({
    'spend': 'sum',
    'revenue': 'sum',
    'conversions': 'sum'
}).reset_index()

channel_daily['roas'] = channel_daily['revenue'] / channel_daily['spend']

# Create interactive plot for channel ROAS trends
fig = px.line(channel_daily, x='date', y='roas', color='channel',
              title='ROAS Trend by Channel Over Time',
              labels={'roas': 'ROAS', 'date': 'Date'},
              height=500)

fig.add_hline(y=1, line_dash="dash", line_color="gray",
              annotation_text="Break-even ROAS")

fig.show()

## 4. Campaign Analysis

In [ ]:
# Campaign performance analysis
campaign_performance = daily_metrics.groupby('campaign').agg({
    'spend': 'sum',
    'revenue': 'sum',
    'conversions': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()

campaign_performance['roas'] = campaign_performance['revenue'] / campaign_performance['spend']
campaign_performance['cpa'] = campaign_performance['spend'] / campaign_performance['conversions']
campaign_performance['conversion_rate'] = campaign_performance['conversions'] / campaign_performance['clicks'] * 100

# Sort by revenue
campaign_performance = campaign_performance.sort_values('revenue', ascending=False)

print("Campaign Performance Summary:")
campaign_performance

In [ ]:
# Campaign performance heatmap
campaign_channel = daily_metrics.groupby(['campaign', 'channel']).agg({
    'roas': 'mean'
}).reset_index()

campaign_channel_pivot = campaign_channel.pivot(index='campaign', columns='channel', values='roas')

plt.figure(figsize=(12, 8))
sns.heatmap(campaign_channel_pivot, annot=True, fmt='.2f', cmap='RdYlGn', center=1,
            cbar_kws={'label': 'ROAS'})
plt.title('ROAS Heatmap: Campaign x Channel', fontsize=16)
plt.xlabel('Channel')
plt.ylabel('Campaign')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Customer Analysis

In [ ]:
# Customer segment analysis
segment_summary = customer_data.groupby('segment').agg({
    'customer_id': 'count',
    'lifetime_value': ['mean', 'sum'],
    'total_orders': 'mean',
    'churn_risk_score': 'mean'
}).reset_index()

segment_summary.columns = ['segment', 'customer_count', 'avg_ltv', 'total_ltv', 'avg_orders', 'avg_churn_risk']
segment_summary['ltv_per_order'] = segment_summary['avg_ltv'] / segment_summary['avg_orders']
segment_summary = segment_summary.sort_values('avg_ltv', ascending=False)

print("Customer Segment Summary:")
segment_summary

In [ ]:
# Customer segment visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Customer distribution
segment_summary.plot(x='segment', y='customer_count', kind='bar', ax=axes[0, 0], color='lightblue')
axes[0, 0].set_title('Customer Distribution by Segment', fontsize=14)
axes[0, 0].set_ylabel('Number of Customers')

# Average LTV by segment
segment_summary.plot(x='segment', y='avg_ltv', kind='bar', ax=axes[0, 1], color='lightgreen')
axes[0, 1].set_title('Average Customer Lifetime Value by Segment', fontsize=14)
axes[0, 1].set_ylabel('Average LTV ($)')

# Churn risk by segment
segment_summary.plot(x='segment', y='avg_churn_risk', kind='bar', ax=axes[1, 0], color='coral')
axes[1, 0].set_title('Average Churn Risk Score by Segment', fontsize=14)
axes[1, 0].set_ylabel('Churn Risk Score')

# LTV vs Orders scatter
axes[1, 1].scatter(segment_summary['avg_orders'], segment_summary['avg_ltv'], 
                   s=segment_summary['customer_count']/10)
for i, txt in enumerate(segment_summary['segment']):
    axes[1, 1].annotate(txt, (segment_summary['avg_orders'].iloc[i], 
                             segment_summary['avg_ltv'].iloc[i]), fontsize=9)
axes[1, 1].set_xlabel('Average Orders')
axes[1, 1].set_ylabel('Average LTV ($)')
axes[1, 1].set_title('Customer Value vs Purchase Frequency', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# Customer acquisition trends
customer_data['acquisition_month'] = customer_data['acquisition_date'].dt.to_period('M')
acquisition_trend = customer_data.groupby(['acquisition_month', 'preferred_channel']).size().reset_index(name='new_customers')
acquisition_trend['acquisition_month'] = acquisition_trend['acquisition_month'].astype(str)

# Create stacked area chart
fig = px.area(acquisition_trend, x='acquisition_month', y='new_customers', color='preferred_channel',
              title='Customer Acquisition Trend by Preferred Channel',
              labels={'new_customers': 'New Customers', 'acquisition_month': 'Month'})
fig.show()

## 6. Device and Behavioral Analysis

In [ ]:
# Device performance analysis
device_metrics = pd.DataFrame({
    'device': ['Mobile', 'Desktop', 'Tablet'],
    'conversions': [
        daily_metrics['mobile_conversions'].sum(),
        daily_metrics['desktop_conversions'].sum(),
        daily_metrics['tablet_conversions'].sum()
    ]
})

device_metrics['conversion_share'] = device_metrics['conversions'] / device_metrics['conversions'].sum() * 100

# Behavioral metrics by channel
behavioral_metrics = daily_metrics.groupby('channel').agg({
    'bounce_rate': 'mean',
    'avg_session_duration': 'mean',
    'pages_per_session': 'mean'
}).reset_index()

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Device distribution
axes[0].pie(device_metrics['conversion_share'], labels=device_metrics['device'], 
            autopct='%1.1f%%', startangle=90, colors=['#ff9999', '#66b3ff', '#99ff99'])
axes[0].set_title('Conversion Distribution by Device', fontsize=14)

# Behavioral metrics heatmap
behavioral_pivot = behavioral_metrics.set_index('channel').T
sns.heatmap(behavioral_pivot, annot=True, fmt='.2f', cmap='YlOrRd', ax=axes[1])
axes[1].set_title('Behavioral Metrics by Channel', fontsize=14)

plt.tight_layout()
plt.show()

## 7. A/B Test Results Analysis

In [ ]:
# A/B test results visualization
print("A/B Test Results:")
ab_test_results

In [ ]:
# Calculate conversion rates and confidence intervals
ab_test_results['variant_a_rate'] = ab_test_results['variant_a_conversions'] / ab_test_results['variant_a_sample_size'] * 100
ab_test_results['variant_b_rate'] = ab_test_results['variant_b_conversions'] / ab_test_results['variant_b_sample_size'] * 100

# Visualize A/B test results
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(ab_test_results))
width = 0.35

bars1 = ax.bar(x - width/2, ab_test_results['variant_a_rate'], width, label='Variant A', color='lightcoral')
bars2 = ax.bar(x + width/2, ab_test_results['variant_b_rate'], width, label='Variant B', color='lightgreen')

ax.set_xlabel('Test Name')
ax.set_ylabel('Conversion Rate (%)')
ax.set_title('A/B Test Results Comparison')
ax.set_xticks(x)
ax.set_xticklabels(ab_test_results['test_name'], rotation=45, ha='right')
ax.legend()

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

# Add significance indicators
for i, row in ab_test_results.iterrows():
    if row['statistical_significance'] > 0.95:
        ax.text(i, max(row['variant_a_rate'], row['variant_b_rate']) + 1, 
                '***', ha='center', va='bottom', fontsize=12, color='green')

plt.tight_layout()
plt.show()

## 8. Key Insights and Recommendations

In [ ]:
# Generate key insights
print("KEY MARKETING INSIGHTS:")
print("=" * 50)

# Top performing channel
top_channel = channel_performance.iloc[0]
print(f"\n1. TOP PERFORMING CHANNEL: {top_channel['channel']}")
print(f"   - ROAS: {top_channel['roas']:.2f}x")
print(f"   - Revenue Share: {top_channel['revenue_share']:.1f}%")
print(f"   - CPA: ${top_channel['cpa']:.2f}")

# Worst performing channel
worst_channel = channel_performance.iloc[-1]
print(f"\n2. UNDERPERFORMING CHANNEL: {worst_channel['channel']}")
print(f"   - ROAS: {worst_channel['roas']:.2f}x")
print(f"   - Recommendation: {'Reduce spend' if worst_channel['roas'] < 1 else 'Optimize targeting'}")

# Best campaign
best_campaign = campaign_performance.iloc[0]
print(f"\n3. BEST CAMPAIGN: {best_campaign['campaign']}")
print(f"   - Revenue: ${best_campaign['revenue']:,.2f}")
print(f"   - ROAS: {best_campaign['roas']:.2f}x")

# Customer insights
vip_segment = segment_summary[segment_summary['segment'] == 'VIP Customers'].iloc[0]
print(f"\n4. VIP CUSTOMERS:")
print(f"   - Average LTV: ${vip_segment['avg_ltv']:.2f}")
print(f"   - Total Value: ${vip_segment['total_ltv']:,.2f}")
print(f"   - Recommendation: Focus retention efforts on this segment")

# Mobile performance
mobile_share = device_metrics[device_metrics['device'] == 'Mobile']['conversion_share'].iloc[0]
print(f"\n5. MOBILE DOMINANCE:")
print(f"   - {mobile_share:.1f}% of all conversions come from mobile")
print(f"   - Recommendation: Prioritize mobile experience optimization")

# Seasonality
best_month = monthly_performance.loc[monthly_performance['revenue'].idxmax()]
print(f"\n6. SEASONALITY:")
print(f"   - Peak month: {best_month['month']}")
print(f"   - Peak revenue: ${best_month['revenue']:,.2f}")
print(f"   - Recommendation: Plan major campaigns for Q4")

In [ ]:
# Budget optimization recommendations
print("\nBUDGET OPTIMIZATION RECOMMENDATIONS:")
print("=" * 50)

total_budget = channel_performance['spend'].sum()
for _, channel in channel_performance.iterrows():
    current_share = channel['spend'] / total_budget * 100
    
    if channel['roas'] > 3:
        recommendation = "↑ Increase budget by 20-30%"
    elif channel['roas'] > 2:
        recommendation = "→ Maintain current budget"
    elif channel['roas'] > 1:
        recommendation = "↓ Decrease budget by 10-15%"
    else:
        recommendation = "⚠️ Consider pausing or major optimization"
    
    print(f"\n{channel['channel']}:")
    print(f"  Current budget share: {current_share:.1f}%")
    print(f"  Current ROAS: {channel['roas']:.2f}x")
    print(f"  Recommendation: {recommendation}")

In [ ]:
# Save summary statistics for dashboard
summary_data = {
    'overall_metrics': {
        'total_spend': total_spend,
        'total_revenue': total_revenue,
        'total_conversions': total_conversions,
        'overall_roas': overall_roas,
        'overall_cpa': overall_cpa
    },
    'top_performers': {
        'best_channel': top_channel['channel'],
        'best_campaign': best_campaign['campaign'],
        'best_segment': 'VIP Customers'
    },
    'optimization_targets': {
        'worst_channel': worst_channel['channel'],
        'high_risk_segment': segment_summary.loc[segment_summary['avg_churn_risk'].idxmax(), 'segment']
    }
}

import json
with open('../data/summary_insights.json', 'w') as f:
    json.dump(summary_data, f, indent=2)

print("\nSummary insights saved to data/summary_insights.json")